# Notebook used for the development of the ImmuneDB Preproccing tools
----

Objectives:
1. [_] Creation of mutations dataframe.
2. [_] Creation of mutations input of the LPA algorithm.
3. [_] Creation of cleaned sequences dataframe.
4. [_] Creation of trimers input for the LPA algorithm
5. [_] Visualization functionality ?
----


> \> Creation of mutations dataframe

In [ ]:
# Import required packages
from source.helpers import read_json, create_folders

from sqlalchemy import create_engine
import importlib.metadata
import pandas as pd
import json
import os


######################################################################################################
# A custom function that connects to MySQL server, execute query and returns the results as dataframe.
class sql_conn():
    def __init__ (self):
        """
        Automaticly takes sql credentials from the `config.json` file:
        username:str -> Username credentials for the MySQL server.
        password:str -> Password credentials for the MySQL server.
        adress:str -> IP adress of the MySQL server.
        port:str -> Port of the MySQL server.
        qry:str -> SQL query to be executed.
        """

        # Setting up sql credentials
        sql_cred, sql_db = read_json()["sql"], read_json()["database"]
        username, password, adress, port = sql_cred["username"], sql_cred["password"], sql_cred["adress"], sql_cred["port"]
        self.db_name = sql_db["db_name"]

        # Setting up MySQL connenction
        connection_mysql = f"mysql+pymysql://{username}:{password}@{adress}:{port}/{self.db_name}"
        self.engine = create_engine(connection_mysql)
        print(f"> Established connecntion to the {self.db_name} database.")

    def get_table(self,
                  table_name,
                  sql_qry : str = "defualt",
                  save_table:bool = True) -> pd.DataFrame:
        """
        qry : str -> string of sql input, to be executed on the sql server.
        save_table : bool -> to save the returened sql table to the raw_folder.
        """
    
        # cheeking for custom query input
        qry_bool = (sql_qry == "defualt")
        query_map = {True : f"SELECT * FROM {self.db_name}.{table_name}",
                     False : sql_qry}

        # Executing the query
        try:
            sql_qry = query_map[qry_bool]
            qry_df = pd.read_sql(sql_qry, self.engine)
            print(f"> Query executed successfully: \n{sql_qry}")
        except:
            raise Exception("> Got error while exeuting the query, please verify syntax.")

        if save_table:
            save_path = os.path.join("tables_raw", self.db_name, f"{self.db_name}_{table_name}.tsv")
            qry_df.to_csv(save_path, sep="\t", index=False)

        return qry_df
    
    # Closing the connenction
    def sql_close(self):
        self.engine.dispose()
        print("> MySQL connenction terminated.")

> folder `tables_output` was created.
> folder `tables_processed` was created.
> folder `tables_raw` was created.
> folder `tables_output\covid_vaccine_new` was created.
> folder `tables_processed\covid_vaccine_new` was created.
> folder `tables_raw\covid_vaccine_new` was created.


In [2]:
sql_connection = sql_conn()
sql_connection.get_table("sample_metadata")

> Established connecntion to the covid_vaccine_new database.
> Query executed successfully: 
SELECT * FROM covid_vaccine_new.sample_metadata


,sample_id,key,value
0,3,date_hic_received,########
1,3,date_sample_received,########
2,4,date_hic_received,########
3,4,date_sample_received,########
4,5,date_hic_received,########
...,...,...,...
1677,62,age_units,years
1678,63,age_units,years
1679,64,age_units,years
1680,65,age_units,years
